# Soribada

## 라이브러리 로드

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import os, time, random
from selenium import webdriver
import chromedriver_autoinstaller as ca

## 준비

### 현재 경로 확인

In [2]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/music'

### 폴더 생성

In [3]:
# 파일 저장할 폴더 생성
crawled_folder_path = code_path + '/crawled_data/week_soribada/'
os.makedirs(crawled_folder_path, exist_ok=True)

### 크롬 드라이브 버전 확인

In [4]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'100'

In [5]:
# 크롬 드라이버 확인 및 설치
try:
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')
except:
    ca.install(True)
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')

C:\Users\kbjung\AppData\Local\Temp/ipykernel_8376/231581722.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')


## 페이지 접속

In [7]:
# 페이지 접속
url = 'https://www.soribada.com/music/chart/weekly/'
driver.get(url)
driver.implicitly_wait(3)
driver.maximize_window()

### 링크 만들기

In [8]:
# 날짜
dates = ['2021124', '2022011', '2022012', '2022013', '2022014', '2022015', '2022021', '2022022', '2022023', '2022024', '2022031', '2022032', '2022033', '2022034']
# 링크용 날짜
link_date_list = []
for one in dates:
    temp = url + one
    link_date_list.append(temp)
len(link_date_list)

14

In [9]:
link_date_list[:5]

['https://www.soribada.com/music/chart/weekly/2021124',
 'https://www.soribada.com/music/chart/weekly/2022011',
 'https://www.soribada.com/music/chart/weekly/2022012',
 'https://www.soribada.com/music/chart/weekly/2022013',
 'https://www.soribada.com/music/chart/weekly/2022014']

In [10]:
soup = bs(driver.page_source, 'lxml')
soup.find('span', 'nowText').text

'2021년 12월 27일 ~ 2022년 01월 02일'

In [11]:
temp = soup.find('span', 'nowText').text.split(' ~ ')[0]
temp = temp.replace('년 ' ,'-')
temp = temp.replace('월 ' ,'-')
temp = temp.replace('일' ,'')
temp

'2021-12-27'

In [12]:
# 일간 랭크용 날짜
repeat_rank_date_list = []
temp = soup.find('span', 'nowText').text.split(' ~ ')[0]
temp = temp.replace('년 ' ,'-')
temp = temp.replace('월 ' ,'-')
temp = temp.replace('일' ,'')
for d in range(100):
    repeat_rank_date_list.append(temp)
len(repeat_rank_date_list)

100

In [13]:
for i in range(len(link_date_list)):
    try:
        # 페이지 접속
        driver.get(link_date_list[i])
        driver.implicitly_wait(3)
        time.sleep( random.uniform(1, 1.5) )

        # 수프 담기
        soup = bs(driver.page_source, 'lxml')

        # 리스트
        li_soup = soup.find_all('li', 'listen')

        # 일간 랭크용 날짜
        repeat_rank_date_list = []
        temp = soup.find('span', 'nowText').text.split(' ~ ')[0]
        temp = temp.replace('년 ' ,'-')
        temp = temp.replace('월 ' ,'-')
        temp = temp.replace('일' ,'')
        for d in range(100):
            repeat_rank_date_list.append(temp)

        # rank
        rank_list = []
        for k in range(1, 101):
            rank_list.append(k)

        song_title_list = []
        artist_list = []
        album_list = []
        for one in li_soup:
            # song title
            song_title = one.find('span', 'song-title').text
            song_title_list.append(song_title)

            # artist
            artist = one.find('div', 'list-area2').find_all('span')[6].text.strip()
            artist_list.append(artist)

            # album
            album = one.find('span', 'link-type2 album-title').text
            album_list.append(album)
        
        # df
        dict = {'날짜':repeat_rank_date_list, '순위':rank_list, '곡':song_title_list, '가수':artist_list, '앨범':album_list}
        df = pd.DataFrame(dict)

        # make excel
        date = dates[i]
        file_name = f'week_soribada_{date}.xlsx'
        df.to_excel(crawled_folder_path + file_name, index=False)

        print(f"{file_name} 파일 생성 완료")
        
    except:
        pass

week_soribada_2021124.xlsx 파일 생성 완료
week_soribada_2022015.xlsx 파일 생성 완료
week_soribada_2022021.xlsx 파일 생성 완료
week_soribada_2022022.xlsx 파일 생성 완료
week_soribada_2022023.xlsx 파일 생성 완료
week_soribada_2022024.xlsx 파일 생성 완료
week_soribada_2022032.xlsx 파일 생성 완료
week_soribada_2022033.xlsx 파일 생성 완료
week_soribada_2022034.xlsx 파일 생성 완료


# 코드 마지막